# Tools
This notebook takes the processed data and makes two tools - a simple Unity Catalog function and a Vector index with SQL vector search function.

In [0]:
%pip install databricks_langchain langgraph==0.3.4 mlflow databricks-vectorsearch
%restart_python

In [0]:
import yaml
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

dbutils.widgets.text("catalog", config['catalog'])
dbutils.widgets.text("schema", config['schema'])
dbutils.widgets.text("products_table", config['products_table'])
dbutils.widgets.text("fixed_camp_table", config['fixed_camp_table'])
dbutils.widgets.text("fm_endpoint_name", config['fm_endpoint_name'])

A simple SQL function as a tool - it has to have a good description and comments to allow our agent to call it properly.

In [0]:
%sql
CREATE OR REPLACE FUNCTION ${catalog}.${schema}.campaign_total(
  template_name STRING COMMENT 'Name of the campaign template, should be Template followed by an integer, e.g. Template 0'
)
RETURNS TABLE(
  total_cost DOUBLE,
  campaign_start DATE,
  campaign_end DATE
  )
COMMENT 'Returns the total cost and dates of a campaign. If the user asks for a campaign number, use Template x, where x is an integer up to 20. If the table is blank it likely means that campaign number does not exist'
RETURN
SELECT 
  SUM(cost) as total_cost,
  MIN(start_date) as campaign_start,
  MAX(end_date) as campaign_end
FROM ${catalog}.${schema}.${fixed_camp_table}
WHERE lower(template) = lower(template_name)

In [0]:
%sql
SELECT * FROM ${catalog}.${schema}.campaign_total('Template 0')

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

## Genie Agent
Next we are going to use the databricks-langchain notebook to do tool creation using a Genie workspace. See https://github.com/databricks/databricks-ai-bridge for more.

Databricks LangChain (e.g. AI Bridge) has a wrapper around the Genie Conversation API that allows the creation of a LangGraph Node or Agent for a Genie Space.

In [0]:
from databricks_langchain.genie import GenieAgent
import mlflow

genie_space_id = config['genie_space_id']
assert genie_space_id is not None, "Please fill out your Genie Space ID!"
genie_agent = GenieAgent(
  genie_space_id,
  "marketing_agent",
  description="""
    This agent can access information about marketing campaigns and their performance. Example questions include
    - How has the total number of emails sent, delivered, and the unique clicks evolved over the last six months?
    - Which industries have shown the highest engagement rates with marketing campaigns?
  """,
  client=w,
),

We can now use the familiar `invoke` pattern that langchain uses

In [0]:
input_example = {
    "messages": [
        {
            "role": "user",
            "content": "What are the top 3 subject lines for my campaigns that led to the most number of opens?",
        },        
    ]
}

genie_agent.invoke(input_example)

## Vector Search
We can also serve Vector Search as either a Langchain Retriever, or a UC Function. My preference right now is the UC Function due to the universal usability in a) Playground, b) ReACT Agents, and c) Agents via tools.

In [0]:
from databricks.vector_search.client import VectorSearchClient
client = VectorSearchClient()

In [0]:
%sql
ALTER TABLE ${catalog}.${schema}.${fixed_camp_table}
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

We can sync or create the vector search using the following code snippet

In [0]:
source_table = f"{config['catalog']}.{config['schema']}.{config['fixed_camp_table']}"
index_name = source_table+"_index"
try:
  index = client.create_delta_sync_index(
    endpoint_name=config['vector_search_endpoint'],
    source_table_name=source_table,
    index_name=index_name,
    pipeline_type="TRIGGERED",
    primary_key="campaign_id",
    embedding_source_column="campaign_description",
    embedding_model_endpoint_name=config['em_endpoint_name']
  )
except Exception as e:
  print(e)
  index = client.get_index(index_name=index_name)
  index.sync()

We can also convert the vector search into a Unity Catalog function and call it in playground, or agentic flows.

In [0]:
%sql
CREATE OR REPLACE FUNCTION ${catalog}.${schema}.campaign_search(
  description STRING COMMENT 'A campaign description'
)
RETURNS TABLE (
  campaign_id INTEGER,
  template STRING,
  campaign_name STRING,
  campaign_description STRING
)
COMMENT 'Returns the top three campaing template, name descriptions based on semantically similarity to the input query.
'
RETURN
SELECT campaign_id, template, campaign_name, campaign_description
FROM vector_search(
  index=>'${catalog}.${schema}.${fixed_camp_table}_index',
  query_text=>description,
  num_results=>3,
  query_type=>'hybrid'
)

In [0]:
%sql
SELECT * 
FROM ${catalog}.${schema}.campaign_search('Viking range hood')